# MOM :

1. **Objetivos**

Análise numérica das propriedades de antenas tipo dipolos usando o Método dos Momentos.
<br/>
<br/>
2. **Geometria**

Seja uma antena de dipolo linear, de comprimento L, ao longo do eixo dos z, alimentada pelo centro. Considere que
o raio do fio a seja muito menor que o comprimento de onda, e que o metal da antena seja perfeitamente condutor.
<br/>
<br/>
3. **Prática**

    1. Seguindo o procedimento mostrado em sala, obtenha os elementos das matrizes de Impedância (Z) e de Tensão (V) para funções de base tipo triangulares.
    2. Implemente a formulação em Matlab ou outro ambitente/linguagem desejada.
    3. Para uma antena de meio comprimento-de-onda ($L = \lambda/2$), e raio $a = 10^{−4}\lambda$, determine a distribuiçãoo de corrente ao longo da antena. Use 19 funções de base. Varie o número de funçõess de base (N) e observe a convergência.
    4. Para a antena do item anterior, obtenha a impedância de entrada, e mostre sua convergência com N.
    5. Repita o item 3 para uma antena de 1 comprimento-de-onda ($L = \lambda$).
    6. Para a antena de ($L = \lambda$), obtenha a impedância de entrada, e mostre sua variação com N.


In [ ]:
#Import
import scipy.constants as sci
import numpy as np
import matplotlib.pyplot as pl
import math
from mpl_toolkits.mplot3d import Axes3D 

from matplotlib import cm
from numpy.linalg import inv
from scipy.linalg import toeplitz

In [ ]:
#define constants
E0 = sci.epsilon_0
#U0 = sci.mu_0
Clight = sci.speed_of_light
Freq = 299792458.0 #Hz
Lambda = Clight/Freq
Pi = math.pi
k0 = 2*Pi/Lambda
pl.rcParams['figure.figsize'] = [10, 7]

In [ ]:
def voltageColumn(N,V0):
    V = np.zeros((N,1),dtype=np.complex_)
    if(N%2==0):
        temp = int(N/2)
        V[temp] = V0;
        V[temp-1]= V0;
    else:
        temp = int((N-1)/2)
        V[temp] = V0
    return V


In [ ]:
def funcZ(x,L,N):
    Delta = L/(N+1)
    if(x < 0):
        x = 0
    return -L/2 + x*Delta

In [ ]:
def funcPsi(m,n, L, N,a):
    Delta = L/(N+1)
    if(m == n):
        fst = 1./(2*Pi*Delta)
        snd = np.log((Delta/2 + np.sqrt( (Delta/2)**2 + a**2  ) ) /a )
        third = 1j*k0/4*Pi
        return fst*snd - third
    else:
        fst = np.sqrt( (funcZ(m,L,N) - funcZ(n,L,N))**2 + a**2 )
        snd = np.exp(-1j*k0*fst)
        third = 4*Pi
        return snd/(third*fst)

In [ ]:
def funcAmn(m,n,L,N,a):
    Delta = L/(N+1)
    return (Delta**2)*funcPsi(m,n,L,N,a)

In [ ]:
def funcPhi(m,n,L,N,a):
    fst = funcPsi(m-1/2 , n-1/2,L,N,a)
    snd = funcPsi(m+1/2 , n-1/2,L,N,a)
    thd = funcPsi(m-1/2 , n+1/2,L,N,a)
    qrt = funcPsi(m+1/2 , n+1/2,L,N,a)
    return (fst - snd - thd + qrt)

In [ ]:
def impedanceMtz(L,N,a):
    Zmn = np.empty([N,N], dtype=np.complex_)
    ZmnAux = np.zeros((1,N), dtype=np.complex_)
    for i in range(0,N):
        for j in range (0,N):
            Zmn[i][j] = (k0**2)*funcAmn(i+1,j+1,L,N,a) - funcPhi(i+1,j+1,L,N,a)
    for i in range(0,N):
        ZmnAux[0][i] = (k0**2)*funcAmn(1,i+1,L,N,a) - funcPhi(1,i+1,L,N,a)
    return Zmn, ZmnAux
            

## Current Approach

***

In [ ]:
def currentDist(L,N,a, V0):
    Zmtz, Zaux = impedanceMtz(L,N,a)
    Zaux = toeplitz(Zaux)
    ZauxInv = inv(Zaux)
    ZInv = inv(Zmtz)
    Vin = voltageColumn(N,V0)
    Imnaux = np.dot(ZauxInv,Vin)
    Imn = np.dot(ZInv,Vin)
    
    return Imn, np.linspace(-L/2, L/2, N+2)

In [ ]:
def plotCurrentDist(L,N,a, V0, strL):
    Imn, vec_N = currentDist(L,N,a,V0)
    zero_pad = np.linspace(0,0,1, dtype=np.complex_)
    test = np.concatenate((zero_pad,Imn), axis=None)
    test = np.concatenate((test,zero_pad), axis=None)
    
    pl.plot(vec_N, 1000*abs(test), label='N = '+ str(N))
    pl.xlabel('z/$\lambda$', fontsize=15)
    pl.ylabel('|Im| (mA)', fontsize=15)
#     pl.title('Distribuição de Corrente para L =' + strL)
    
    pl.legend(loc='best', shadow=True, fontsize='large')

In [ ]:
L = Lambda/2
N = 41
a = Lambda*(10**(-4))
V0 = -1j*2*Pi*Freq*E0
plotCurrentDist(L,5,a,V0, "$\lambda$/2")
plotCurrentDist(L,19,a,V0, "$\lambda$/2")
plotCurrentDist(L,61,a,V0, "$\lambda$/2")
pl.savefig('disthalf.png')

In [ ]:
L = Lambda
N = 41
a = Lambda*(10**(-4))
V0 = -1j*2*Pi*Freq*E0
plotCurrentDist(L,9,a,V0, "$\lambda$")
plotCurrentDist(L,19,a,V0, "$\lambda$")
plotCurrentDist(L,61,a,V0, "$\lambda$")
pl.savefig('distfull.png')

In [ ]:
def entryCurrent(L,N,a,V0):
    Zmtz, Zaux = impedanceMtz(L,N,a)
    Zaux = toeplitz(Zaux)
    ZauxInv = inv(Zaux)
    ZInv = inv(Zmtz)
    Vin = voltageColumn(N,V0)
    Imnaux = np.dot(ZauxInv,Vin)
    Imn = np.dot(ZInv,Vin)
    return Imn[int((N-1)/2)]

In [ ]:
def allCurrent(itr,L,a,V0):
    list_convert = []
    for i in range(3,itr,4):
        list_convert.append( entryCurrent(L,i,a,V0) )
    return np.array(list_convert,dtype=np.complex_), np.arange(3,itr,4)

In [ ]:
def plotAllCurrent(itr,L,a,V0,strL):
    test_vec1, test_vec2 = allCurrent(itr,L, a, V0 )
    # Points
    pl.plot(test_vec2, 1000*abs(test_vec1), linestyle='--', marker='o', color='b')

    # Show the major grid lines with dark grey lines
    # pl.grid(b=True, linestyle="-", color='black', which='major')

    # Show the minor grid lines with very faint and almost transparent grey lines
    pl.minorticks_on()
    pl.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.5)

    pl.xlabel('Segmentos', fontsize=15)
    pl.ylabel('|Im| (mA)', fontsize=15)
#     pl.title('Convergência da Corrente $Im(0)$ para L = ' + strL)

In [ ]:
limit = 100
L = Lambda/2
a = Lambda*(10**(-4))
V0 = -1j*2*Pi*Freq*E0
plotAllCurrent(limit, L,a,V0, "$\lambda$/2")
pl.savefig('convhalf.png')

In [ ]:
limit = 100
L = Lambda
a = Lambda*(10**(-4))
V0 = -1j*2*Pi*Freq*E0
plotAllCurrent(limit,L,a,V0 , "$\lambda$")
pl.savefig('convfull.png')

## Impedance Approach

***

In [ ]:
def entryImpedance(L,N,a,V0):
    Zmtz, Zaux = impedanceMtz(L,N,a)
    Zaux = toeplitz(Zaux)
    ZauxInv = inv(Zaux)
    ZInv = inv(Zmtz)
    Vin = voltageColumn(N,V0)
    Imnaux = np.dot(ZauxInv,Vin)
    Imn = np.dot(ZInv,Vin)
    return 1./Imn[int((N-1)/2)]

In [ ]:
def allImpedance(itr,L,a,V0):
    list_convert = []
    for i in range(3,itr,4):
        list_convert.append( entryImpedance(L,i,a,V0) )
        
    return np.array(list_convert, dtype=np.complex_), np.arange(3,itr,4)

In [ ]:
def plotAllImpedance(itr, L,a,V0, strL):
    test_vec1, test_vec2 = allImpedance(itr,L, a, V0 )
    # Points
    pl.plot(test_vec2, test_vec1.real, linestyle='--', marker='o', color='b', label = '$R_{in} (\Omega)$')
    pl.plot(test_vec2, test_vec1.imag, linestyle='--', marker='o', color='r', label = '$X_{in} (\Omega)$ ')
    pl.legend(loc='center', shadow=True, fontsize='large')

    # Show the major grid lines with dark grey lines
#     pl.grid(b=True, linestyle="-", color='black', which='major')

    # Show the minor grid lines with very faint and almost transparent grey lines
    pl.minorticks_on()
    pl.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.5)

    pl.xlabel('Segmentos', fontsize=15)
    pl.ylabel('Ohms ($\Omega$)', fontsize=15)
#     pl.title('Convergência de $Z_{in}$ para L = ' + strL)
    

In [ ]:
limit = 100
L = Lambda/2
a = Lambda*(10**(-4))
V0 = -1j*2*Pi*Freq*E0
plotAllImpedance(limit,L,a,V0 , "$\lambda$/2")
pl.savefig('zinhalf.png')

In [ ]:
limit = 100
L = Lambda
a = Lambda*(10**(-4))
V0 = -1j*2*Pi*Freq*E0
plotAllImpedance(limit,L,a,V0 ,"$\lambda$")
pl.savefig('zinfull.png')

In [ ]:
def makeTable(L,a,V0):
    print("N\t\t\tIm(0)\t\t\tZin")
    for i in range(3,100,2):
        print (i, "\t", 1000*entryCurrent(L,i,a,V0), "\t", entryImpedance(L,i,a,V0))

In [ ]:
L = Lambda/2
a = Lambda*(10**(-4))
V0 = -1j*2*Pi*Freq*E0
makeTable(L,a,V0)

In [ ]:
L = Lambda
a = Lambda*(10**(-4))
V0 = -1j*2*Pi*Freq*E0
makeTable(L,a,V0)